In [1]:
import pandas as pd
import random
import os
import numpy as np
from datetime import datetime
import multiprocessing as mp
import time
import re

In [2]:
DATA_DIR = "../Data"
TWEETS_PATH = os.path.join(DATA_DIR, 'tweets')
TREND_PATH = os.path.join(DATA_DIR, 'all_trends_world.csv')
SAVE_PATH = os.path.join(DATA_DIR, 'save')
os.listdir(DATA_DIR)

['2019-08-31_tweetsevenmorebasic.csv',
 'tweets',
 'save',
 'all_trends_world.csv']

In [10]:
def camel_case_split(onegram):
    match_list = []
    for identifier in set(onegram):    
        matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
        match_list += [m.group(0) for m in matches]
    
    return match_list
        
def onegram_augment(onegram):
    
    onegram = set(onegram)
    onegram_up = set([gram.upper() for gram in onegram])
    onegram_lower = set([gram.lower() for gram in onegram])
    
    nohash = set([re.sub('#','', gram) for gram in onegram])
    nohash_up = set([gram.upper() for gram in nohash])
    nohash_lower = set([gram.lower() for gram in nohash])
    
    camelCase = camel_case_split(nohash) 
    camelSplit = set()
    if len(camelCase) !=0 :
        cc_up = [gram.upper() for gram in camelCase]
        cc_lower = [gram.lower() for gram in camelCase]

        ccHashed = set(['#'+gram for gram in camelCase])
        ccHashed_up = set(['#'+gram for gram in cc_up])
        ccHashed_lower = set(['#'+gram for gram in cc_lower])
        
        ccHashJoined = set(['#'.join(camelCase)])
        ccHashJoined_up = set(['#'.join(cc_up)])
        ccHashJoined_lower = set(['#'.join(cc_lower)])
        
        
        if len(camelCase) > 1:
            camelSplit = camelSplit.union(set([' '.join(camelCase)]), 
                                          set([' '.join(cc_up)]), set([' '.join(cc_lower)]))
        
        camelCase = set().union(ccHashed, ccHashed_up, ccHashed_lower,
                        ccHashJoined_up, ccHashJoined_lower, ccHashJoined)
    
    return (onegram.union(onegram_up,onegram_lower, nohash,nohash_up,nohash_lower,camelCase), camelSplit)


def nonegram_augment(nonegram):
    nonegram = set(nonegram)
    nonegram_up = set([gram.upper() for gram in nonegram])
    nonegram_lower = set([gram.lower() for gram in nonegram])
    
    return nonegram.union(nonegram_up, nonegram_lower)

In [27]:
def index_trends(text, onegram_trend_set, nonegram_trend_set):
    try:
        tokens = text.split(' ')
        trend_set = set()
        
        ####### Match not only the onegram but with augmented set of it  #########
        for onegram, onegram_aug in onegram_trend_set.items():
            onegram_match = set(tokens).intersection(onegram_aug) 
#             print(onegram_augmented)
#             print(tokens)
#             print(camel_split)
            if len(onegram_match)!= 0:
                trend_set.add(onegram)
        
        ####### Match not only the nonegram but with augmented set of it  #########
#         for nonegram, nonegram_aug in nonegram_trend_set.items():
#             others = set([other for other in nonegram_aug if (" " + other + " ") in (" " + text +" ")])
#             if len(others)!=0:
#                 trend_set.add(nonegram)

        return trend_set
    
    except:
        print(text)
        return set()
    

In [28]:
def expand_trend_set(df, trend_col):
    
    non_list_cols = [col for col in (df.columns) if col != trend_col ]
    df2 = pd.DataFrame(df[trend_col].tolist(), index=[df[col] for col in non_list_cols])\
                    .stack()\
                    .reset_index(name=trend_col)[non_list_cols+[trend_col]]
    return df2

In [29]:
def prepare_data_trend_date_indexed_function(file, candidates):
    
    tweets_folder =  TWEETS_PATH
    save_folder = SAVE_PATH
    
    df = pd.read_csv('%s/%s' % (tweets_folder, file))
    dfs = []
    
    for candidate in candidates:
        df_that_day = pd.DataFrame(df)
        trends_that_day = set(trends[trends.date == candidate]['name'])
        
        if (len(trends_that_day) == 0):
            print('trends for %s not found!' % candidate)
            continue

        ################################### AUGMENT TREND SETS ############################################
        trends_that_day_onegrams = set([trend for trend in trends_that_day if len(trend.split(' ')) == 1])
        onegram_trend_set = dict()
        camel_split_set = dict()
        for k in trends_that_day_onegrams:
            v1, v2 = onegram_augment([k])
            onegram_trend_set[k] = v1
            if len(v2)!=0:
                camel_split_set[k] = v2
            
            
        trends_that_day_nonegrams = trends_that_day - trends_that_day_onegrams
        nonegram_trend_set = dict((k, nonegram_augment([k])) for k in trends_that_day_nonegrams )
        #nonegram_trend_set.update(camel_split_set)
        
        ################################### APPLY TREND INDEX #############################################
        
        df_that_day['trends'] = df_that_day.text.apply( lambda x: 
                                            index_trends(x, onegram_trend_set, nonegram_trend_set))
        df_that_day = expand_trend_set(df_that_day, 'trends')
        
        ###################################################################################################
        
        df_that_day['trend_date'] = candidate
        dfs.append(df_that_day)
        
    dfs = pd.concat(dfs)
    new_file = file.split('_')[0] + "_trends.csv"
    dfs.to_csv('%s/%s' % (save_folder, new_file), index=False)


In [7]:
def prepare_data_trend_date_indexed_parallelized():
    
    tweets_folder =  TWEETS_PATH
    save_folder = SAVE_PATH

    files = os.listdir(tweets_folder)
    files = [file for file in files if file >= '2019-07-01' and 'csv' in file] # trends only available after this date
    pool = mp.Pool(mp.cpu_count() - 2)
    
    for i, file in enumerate(files):
        print('%d / %d - %s' % (i, len(files), file))
        date = file.split('_')[0]
        that_day = pd.Timestamp(date).date()
        one_day_before = that_day - pd.Timedelta(days = 1)
        one_day_after = that_day + pd.Timedelta(days = 1)
        candidates = [str(that_day), str(one_day_before), str(one_day_after)]
        pool.apply_async(prepare_data_trend_date_indexed_function, args=(file, candidates))

    pool.close()
    pool.join()
    

In [8]:
def trend_date_parser(d):
    format_in =  "%Y-%m-%d %X"
    format_out = "%Y-%m-%d"
 
    d = datetime.strptime(d, format_in)
    return d.strftime(format_out)

trend_date_parser("2013-07-07 23:36:32")

'2013-07-07'

In [9]:
trends = pd.read_csv(TREND_PATH, parse_dates=['date'], date_parser=trend_date_parser)
tr31 = set(trends[trends.date == '2019-08-31']['name'])
trends[trends.date == '2019-08-31']

,date,duration,name,volume
4173506,2019-08-31,490.0,#الهلال_الرايد,162430.0
4173507,2019-08-31,190.0,코엑스,10596.0
4173508,2019-08-31,100.0,#FoodCPNxTEMPT,99623.0
4173509,2019-08-31,100.0,#あすかなBonDance,27517.0
4173510,2019-08-31,100.0,#اكثر_مطعم_تحبه,0.0
...,...,...,...,...
4174082,2019-08-31,30.0,#Newwieeอยู่นี่,0.0
4174083,2019-08-31,30.0,#استعداداتكم_للمدرسه,0.0
4174084,2019-08-31,30.0,Bill O'Brien,0.0
4174085,2019-08-31,30.0,#ourflowerkai,0.0


In [ ]:
#df = pd.read_csv('%s/%s' % (TWEETS_PATH, '2019-08-31_tweetsevenmorebasic.csv.bz2.bz2'))

In [11]:
prepare_data_trend_date_indexed_parallelized()

0 / 3 - 2019-09-01_tweetsevenmorebasic.csv.bz2.bz2
1 / 3 - 2019-08-30_tweetsevenmorebasic.csv.bz2.bz2
2 / 3 - 2019-08-31_tweetsevenmorebasic.csv.bz2.bz2
@ConnieSchultz If only they'd focused on domestic terrorism in 2009 after recognizing the problem.  Authorities ann… https://t.co/nhGh9Z2PMc
Thalapathians plz concentrate on our tag only #Verithanam   Ithu neraiya peroda Hard work ..Don't spoil it by using… https://t.co/yaew6qrUw4


Process ForkPoolWorker-5:
Process ForkPoolWorker-4:


RT @_Badassiee: half introvert, half wit the shits.


Traceback (most recent call last):
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
  

KeyboardInterrupt: 

In [26]:
onegram = set( [t for t in tr31 if len(t.split(' '))==1 ])
onegram.add("#EzgiYuceturk")
nonegram = tr31 - onegram

onegram_trend_set = dict()
camel_split_set = dict()

for k in onegram:
    v1, v2 = onegram_augment([k])
    onegram_trend_set[k] = v1
    camel_split_set[k] = v2

nonegram_trend_set = dict((k, nonegram_augment([k])) for k in nonegram )
nonegram_trend_set.update(camel_split_set)


In [25]:
#print(df.text[665])
index_trends("My life sucks ezgi yuceturk, forzajuve", onegram_trend_set , nonegram_trend_set)

{'#ForzaJuve'}